## law_1.docx
- pinecone store 저장
    - index name 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = Docx2txtLoader('law_1.docx')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

llm = ChatOpenAI()


c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='law1',
# )
database = PineconeVectorStore(
    index=pc.Index('law1'),
    embedding=embedding,
)

In [4]:
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {'context': database.as_retriever() | format_docs,
     'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("전세사기피해자 금융지원에 대해 설명해주세요.")


'국가는 전세사기피해자 및 임차인의 긴박한 주거안정을 위해 자금을 융자하거나 필요한 지원을 할 수 있으며, 임차인의 경우 「주택도시기금법」에 따라 필요한 자금을 융자받을 수 있습니다. 전세사기피해자 및 임차인에 대한 금융지원 사항은 전세사기피해자법 제27조에서 상세히 규정되어 있습니다.'